## This is the final python script for loading and testing of data

In [111]:
import serial
import sklearn
import pickle
from sklearn.model_selection import train_test_split
import numpy as np
from datetime import datetime

k = 10
banks_filled = 5


port = serial.Serial("COM4", 1324800)

def scale_and_cast(data):
    for i in range(0,data.shape[1]):
        sf = 255/(max(data[:,i]) - min(data[:,i]))
        data[:,i] = (data[:,i] - min(data[:,i])) * sf

    return np.rint(data)

with open('data.pickle', 'rb') as handle:
    data = pickle.load(handle)

X = data["values"][:3000]
y = data["labels"][:3000]

print(len(X))

x_train, x_test, y_train, y_test = train_test_split(scale_and_cast(X), y, test_size=int(len(X)-(banks_filled*64)-k))

x_first = x_train[:k]
y_first = y_train[:k]

x_to_load = x_train[k:]
y_to_load = y_train[k:]


print("Ready to send first points")

for val, lab in zip(x_first,y_first):
    #wait for signal
    signal = port.readline()    


    val_list = [int(x) for x in val]
    val_list.append(int(lab))
    #send data
    port.write(bytearray(val_list))

print("Ready to send training data")

for val, lab in zip(x_to_load,y_to_load):
    #wait for signal
    signal = port.readline()    


    val_list = [int(x) for x in val]
    val_list.append(int(lab))
    #send data
    port.write(bytearray(val_list))

print("Ready to send test data")

program_read_times = []
data_send_times = []
data_received_times = []



for val, label in zip(x_test, y_test):
    #wait for signal
    signal = port.readline()
    program_read_times.append(datetime.now())

    val_list = [int(x) for x in val]
    #send data
    port.write(bytearray(val_list))
    data_send_times.append(datetime.now())

    signal = port.readline()
    data_received_times.append(datetime.now())


data = {"program_read_times":program_read_times,"data_send_times":data_send_times,"data_received_times":data_received_times}

with open("./data/"+str(k)+"_"+str(banks_filled)+".pickle", 'wb') as handle:
    pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

3000
Ready to send first points
Ready to send training data
Ready to send test data
